![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Financial Text Splitting

# 1. Splitting sentences in smaller pieces for token classification purposes (e.g, NER)

##  1.1. Background

Legal languages are usually `verbose`. Sentences have much more words than in conversational or informal versions of the language. Just take a look at this piece of text, extracted from a SEC 10-K filing.

> _Under Nevada law, we are prohibited from paying dividends if the distribution would result in our Company not being able to pay its debts as they become due in the normal course of business if our total assets would be less than the sum of our total liabilities plus the amount that would be needed to pay the dividends, or if we were to be dissolved at the time of distribution to satisfy the preferential rights upon dissolution of stockholders whose preferential rights are superior to those receiving the distribution._

Often times, you can find inline lists:

> _In addition to these objective standards, the NYSE American may delist the securities of any issuer (i) if, in its opinion, the issuer’s financial condition and/or operating results appear unsatisfactory; (ii) if it appears that the extent of public distribution or the aggregate market value of the security has become so reduced as to make continued listing on the NYSE American inadvisable; (iii) if the issuer sells or disposes of principal operating assets or ceases to be an operating company; (iv) if an issuer fails to comply with the NYSE American’s listing requirements; (v) if an issuer’s securities sell at what the NYSE American considers a “low selling price” which the exchange generally considers $0.20 per share and the issuer fails to correct this via a reverse split of shares after notification by the NYSE American; or (vi) if any other event occurs or any condition exists which makes continued listing on the NYSE American, in its opinion, inadvisable._

And sometimes, very long enumerations...

> _On June 20, 2019, Inuvo entered into an Agreement and Plan of Merger Termination Agreement (the “Merger Termination Agreement”) with ConversionPoint Technologies Inc., a Delaware corporation (“CPT”), ConversionPoint Holdings, Inc., a Delaware corporation (“Parent”), CPT Merger Sub, Inc., a Delaware corporation, (“CPT Merger Sub”), and CPT Cigar Merger Sub, Inc., a Nevada corporation (“Inuvo Merger Sub”)_

... and what is worse, some of them combined.

> _On June 20, 2019, Inuvo entered into an Agreement and Plan of Merger Termination Agreement (the “Merger Termination Agreement”) with ConversionPoint Technologies Inc., a Delaware corporation (“CPT”), ConversionPoint Holdings, Inc., a Delaware corporation (“Parent”), CPT Merger Sub, Inc., a Delaware corporation, (“CPT Merger Sub”), and CPT Cigar Merger Sub, Inc., a Nevada corporation (“Inuvo Merger Sub”) which, among other things, terminated the Agreement and Plan of Merger, dated November 2, 2018, by and among Inuvo, CPT, Parent, CPT Merger Sub, and Inuvo Merger Sub, as amended (the “Merger Agreement”), pursuant to which Inuvo would have merged with and into Inuvo Merger Sub and become a wholly-owned subsidiary of Parent, and CPT would have merged with and into CPT Merger Sub and become a wholly-owned subsidiary of Parent (the “Mergers”), and (2) terminated each of the Support Agreements that were entered into by certain officers and directors of Inuvo and the parties to the Merger Agreement_

If reading those long sentences is already a challenge for humans, just imagine how challenging it may be for a machine. Let's talk about these restrictions.

## 1.2. About Out of Vocabulary and Subwording in Tokenization
The number of words in a vocabulary used in a transformer can't be infinite. Better to say, the number of `tokens`. A token is the smallest piece of text we woprk with, obtained after a `tokenization` process. 

`Tokenizing` means separating a sentence into these smaller pieces. In languages with words separated by white spaces, there are at least 3 ways to do that:

![1.jpg](/files/FINLEG/1.jpg)

Let's simplify and think we just separate the tokens by using punctuation. In a sentence like:
`Let's tokenize! Isn't this easy?`

We would be getting:
`Let ' s tokenize ! Isn ' t this easy ?`

That's already 11 tokens to go to the vocabulary. The traditional way to build a vocabulary is was just that:
1) Tokenizing the corpus;
2) Getting each token;
3) Adding it to the vocabulary

**Problem**: languages may consist of 500K to 1M words, if we take into consideration different forms (is not, is, isn ' t ...) . That's something unbearable for real-time NLP architectures to cope with.

### 1.2.1. Out of Vocabulary

**1st solution**: OOV (Out of Vocabulary). The first ML/DL algorithms used to prune the vocabulary, leaving the top-N most frequent tokens seen in the corpus and discarding the rest

This approach soon run short, as the necesities to capture all the meaning of a text without discardign anything became crucial

### 1.2.2. Subword tokenization

The **2nd solution** is _subword tokenization_. The idea behind this is that we can split words into sequences of characters, if we see a probability of that sequence of characters being frequently used to form other words. For example, let's suppose we have the following sentence:

`My favourite Natural Language Processing library is clearly Spark NLP!`

The subword tokenization algorithm used by `BERT` transformers is called WordPiece. The result WordPiece returns for that sentence is:

```
['My', 'favourite', 'Natural', 'Language', 'Process', '##ing', 'library', 'is', 'clearly', 'Spa', '##rk', 'NL', '##P', '!']
```

As we can see, we got:
- `Processing` split into 2 subwords: `Process` and `##ing`. That's because `Process` seems a very common subword in the texts used to train BERT, and `ing` a very common subword (suffix) as well. The `##` means that subwords goes attached to the previous word (`Process`).
- `Spark` split into `Spa`(again, `Spa` is a common subword, for example, `Spa` in Wellness) and `##rk`
- `NLP` into `NL` and `#P`

As you have probably noticed, proper nouns are usually split in more than one subword, as also derivative words (Process - Process+ing).

This second approach is the most used in State of the Art NLP architectures, including in all the Spark NLP Transformers.

### 1.2.3. Length restriction of Transformers

If we thought that our legal sentences were already **long** by themselves, just think about how long they get after we carry out `subword` tokenization.

Transformers have restrictions in terms of the number of (subworded) tokens they can process, as shown in the list below. Any subsequent token after that limit of subwords will be discarded from the sentence.

| Transformer | Limit of tokens |
|-------------|-----------------|
| BERT        | 512             |
| XLNet       | 512             |
| RoBERTa     | 512             |
| XLM-RoBERTa | 512             |
| Electra     | 512             |
| Longformers*| 4096 (8 x BERT) |

As you can see, most of the transformers work with a token limit of 512. There are several exceptions, as Longformer and BigBird, which allow up to 4096.

## 1.3. Conclusions
In this notebook we will show you several techniques and their caveats to properly split Sentences so that you reduce your sentence length after tokenization.

Doing this has proved to be better even when you are within the limits of your sentence token length (512) - smaller chunks of pieces of text are better managed than long sequences.

In [0]:
from johnsnowlabs import * 

# 1. Sentence Splitting for Token Classification

There are two annotators you can find in Spark NLP:
- **SentenceDetector**: a configurable, Rule-based sentence splitter.
- **SentenceDetectorDL**: a trainable, DL-based sentence splitter.

We will be focusing on how to test different splitting rule mechanisms with SentenceDetector, but you can also train your own SentenceDetectorDL by providing with manually split sentences (check the [SentenceDetectorDLApproach](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/9.SentenceDetectorDL.ipynb))

## Non-aggressive sentence splitting
Let's use several characters which don't drastically change the structure of the sentence. Check the documentation of the [SentenceDetector](https://nlp.johnsnowlabs.com/api/python/reference/autosummary/sparknlp/annotator/sentence/sentence_detector/index.html#sparknlp.annotator.sentence.sentence_detector.SentenceDetector).

In [0]:
text = """In addition to these objective standards, the NYSE American may delist the securities of any issuer (i) if, in its opinion, the issuer’s financial condition and/or operating results appear unsatisfactory; (ii) if it appears that the extent of public distribution or the aggregate market value of the security has become so reduced as to make continued listing on the NYSE American inadvisable; (iii) if the issuer sells or disposes of principal operating assets or ceases to be an operating company; (iv) if an issuer fails to comply with the NYSE American’s listing requirements; (v) if an issuer’s securities sell at what the NYSE American considers a “low selling price” which the exchange generally considers $0.20 per share and the issuer fails to correct this via a reverse split of shares after notification by the NYSE American; or (vi) if any other event occurs or any condition exists which makes continued listing on the NYSE American, in its opinion, inadvisable. This is another sentence."""

For this case, just and additional tokenization by `;` with `.setCustomBounds` may suffice.

`.setCustomBounds`accepts a list of regular expressions to be used to split the sentence. By default, already containst he following:

```
Lists (“(i), (ii)”, “(a), (b)”, “1., 2.”)
Numbers
Abbreviations
Punctuations
Multiple Periods
Geo-Locations/Coordinates (“N°. 1026.253.553.”)
Ellipsis (”…”)
In-between punctuations
Quotation marks
Exclamation Points
Basic Breakers (“.”, “;”)
```

If you want to disable predefined expresions, use `.setUseCustomBoundsOnly(True)`. We will leave them.

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")\
    .setCustomBounds([";"])\
    .setUseCustomBoundsOnly(False)

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


In [0]:
lp = LightPipeline(model)

In [0]:
lp.annotate(text)['sentence']

Out[5]: ['In addition to these objective standards, the NYSE American may delist the securities of any issuer',
 '(i) if, in its opinion, the issuer’s financial condition and/or operating results appear unsatisfactory',
 '(ii) if it appears that the extent of public distribution or the aggregate market value of the security has become so reduced as to make continued listing on the NYSE American inadvisable',
 '(iii) if the issuer sells or disposes of principal operating assets or ceases to be an operating company',
 '(iv) if an issuer fails to comply with the NYSE American’s listing requirements',
 '(v) if an issuer’s securities sell at what the NYSE American considers a “low selling price” which the exchange generally considers $0.20 per share and the issuer fails to correct this via a reverse split of shares after notification by the NYSE American',
 'or',
 '(vi) if any other event occurs or any condition exists which makes continued listing on the NYSE American, in its opinion, inadvisable.',
 'This is another sentence.']

For this case, additional tokenization on `;`, `:` with `.setCustomBounds` may suffice

In [0]:
text = 'Taiwan. Speaking only hours after Chinese state media said the time was right to engage in political talks with Taiwan, Foreign Ministry spokesman Shen Guofang told Reuters: "The necessary atmosphere for the opening of the talks has been disrupted by the Taiwan authorities."'

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")\
    .setCustomBounds([";",":"])\
    .setUseCustomBoundsOnly(False)

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


However, several quotes in a sentence may require a splitting by quotes. `;`, `"` with `.setCustomBounds`

In [0]:
text = 'Taiwan. Speaking only hours after Chinese state media said the time was right to engage in political talks with Taiwan, Foreign Ministry spokesman Shen Guofang told Reuters: "The necessary atmosphere for the opening of the talks has been disrupted by the Taiwan authorities.", and "there is no way we can restablish the communication in the next few months"'

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")\
    .setCustomBounds([";","\""])\
    .setUseCustomBoundsOnly(False)

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


In [0]:
lp = LightPipeline(model)

In [0]:
lp.annotate(text)['sentence']

Out[11]: ['Taiwan.',
 'Speaking only hours after Chinese state media said the time was right to engage in political talks with Taiwan, Foreign Ministry spokesman Shen Guofang told Reuters:',
 'The necessary atmosphere for the opening of the talks has been disrupted by the Taiwan authorities.',
 ', and',
 'there is no way we can restablish the communication in the next few months']

## Aggressive sentence-splitting
Sometimes the splitting with "conservative" techniques is not enough. In that case, you may considering breaking the sentence structure. 

This has the main caveat that you may lose context by breaking the sentence into smaller pieces.

However, if there is no other way you can split a sentence, it's better to lose some context but using all the words, than just not splitting and cutting off the sentence on the 512nd token.

Even when you are within the limits, sometimes splitting into smaller chunks helps the model repetitive patterns which may get deluded in big sentences.

Also, the performance degrades the longer the sentence is. Sometimes splitting in smaller sentences may provide with an performance improvement.

In [0]:
text = 'On June 20, 2019, Inuvo entered into an Agreement and Plan of Merger Termination Agreement (the “Merger Termination Agreement”) with ConversionPoint Technologies Inc., a Delaware corporation (“CPT”), ConversionPoint Holdings, Inc., a Delaware corporation (“Parent”), CPT Merger Sub, Inc., a Delaware corporation, (“CPT Merger Sub”), and CPT Cigar Merger Sub, Inc., a Nevada corporation (“Inuvo Merger Sub”) which, among other things, terminated the Agreement and Plan of Merger, dated November 2, 2018, by and among Inuvo, CPT, Parent, CPT Merger Sub, and Inuvo Merger Sub, as amended (the “Merger Agreement”), pursuant to which Inuvo would have merged with and into Inuvo Merger Sub and become a wholly-owned subsidiary of Parent, and CPT would have merged with and into CPT Merger Sub and become a wholly-owned subsidiary of Parent (the “Mergers”), and (2) terminated each of the Support Agreements that were entered into by certain officers and directors of Inuvo and the parties to the Merger Agreement"'

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")\
    .setCustomBounds([";","\""])\
    .setUseCustomBoundsOnly(False)

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


In [0]:
lp = LightPipeline(model)

Conservative splitting does not help

In [0]:
lp.annotate(text)['sentence']

Out[15]: ['On June 20, 2019, Inuvo entered into an Agreement and Plan of Merger Termination Agreement (the “Merger Termination Agreement”) with ConversionPoint Technologies Inc., a Delaware corporation (“CPT”), ConversionPoint Holdings, Inc., a Delaware corporation (“Parent”), CPT Merger Sub, Inc., a Delaware corporation, (“CPT Merger Sub”), and CPT Cigar Merger Sub, Inc., a Nevada corporation (“Inuvo Merger Sub”) which, among other things, terminated the Agreement and Plan of Merger, dated November 2, 2018, by and among Inuvo, CPT, Parent, CPT Merger Sub, and Inuvo Merger Sub, as amended (the “Merger Agreement”), pursuant to which Inuvo would have merged with and into Inuvo Merger Sub and become a wholly-owned subsidiary of Parent, and CPT would have merged with and into CPT Merger Sub and become a wholly-owned subsidiary of Parent (the “Mergers”), and (2) terminated each of the Support Agreements that were entered into by certain officers and directors of Inuvo and the parties to the Merger Agreement']

Splitting by parethensis

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")\
    .setCustomBounds(["\(","\)"])\
    .setUseCustomBoundsOnly(False)

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


In [0]:
lp = LightPipeline(model)

In [0]:
lp.annotate(text)['sentence']

Out[18]: ['On June 20, 2019, Inuvo entered into an Agreement and Plan of Merger Termination Agreement',
 'the “Merger Termination Agreement”',
 'with ConversionPoint Technologies Inc., a Delaware corporation',
 '“CPT”',
 ', ConversionPoint Holdings, Inc., a Delaware corporation',
 '“Parent”',
 ', CPT Merger Sub, Inc., a Delaware corporation,',
 '“CPT Merger Sub”',
 ', and CPT Cigar Merger Sub, Inc., a Nevada corporation',
 '“Inuvo Merger Sub”',
 'which, among other things, terminated the Agreement and Plan of Merger, dated November 2, 2018, by and among Inuvo, CPT, Parent, CPT Merger Sub, and Inuvo Merger Sub, as amended',
 'the “Merger Agreement”',
 ', pursuant to which Inuvo would have merged with and into Inuvo Merger Sub and become a wholly-owned subsidiary of Parent, and CPT would have merged with and into CPT Merger Sub and become a wholly-owned subsidiary of Parent',
 'the “Mergers”',
 ', and',
 '2',
 'terminated each of the Support Agreements that were entered into by certain officers and directors of Inuvo and the parties to the Merger Agreement"']

We can use **REGEX** or just several characters in a row in our `setCustomBounds`to be able to have finegrain splits.

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")\
    .setUseCustomBoundsOnly(False)\
    .setCustomBounds([", and"])

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


In [0]:
lp = LightPipeline(model)

In [0]:
lp.annotate(text)['sentence']

Out[21]: ['On June 20, 2019, Inuvo entered into an Agreement and Plan of Merger Termination Agreement (the “Merger Termination Agreement”) with ConversionPoint Technologies Inc., a Delaware corporation (“CPT”), ConversionPoint Holdings, Inc., a Delaware corporation (“Parent”), CPT Merger Sub, Inc., a Delaware corporation, (“CPT Merger Sub”)',
 'CPT Cigar Merger Sub, Inc., a Nevada corporation (“Inuvo Merger Sub”) which, among other things, terminated the Agreement and Plan of Merger, dated November 2, 2018, by and among Inuvo, CPT, Parent, CPT Merger Sub',
 'Inuvo Merger Sub, as amended (the “Merger Agreement”), pursuant to which Inuvo would have merged with and into Inuvo Merger Sub and become a wholly-owned subsidiary of Parent',
 'CPT would have merged with and into CPT Merger Sub and become a wholly-owned subsidiary of Parent (the “Mergers”)',
 '(2) terminated each of the Support Agreements that were entered into by certain officers and directors of Inuvo and the parties to the Merger Agreement"']

# 2. Merging sentences together for Paragraph / Clause classification

## 2.1. Background
The opposite use case is when you want to carry out Text Classification but only of certain pieces of the document. A good example of it it's Clauses Extraction or Clauses Identification in Legal Documents.

There are several ways we can carry out Text Classification:

1) `At a whole-text level`: That's not feasible for most legal documents. As we already know, we have a 512 token estriction for most BERT-based thansformers. We can use Longformers or BigBird, but in most cases, Legal documents have many pages, what will go for sure beyond any limitation.

2) `Using sentence-splitting`: That will definitely split the document at smaller chunks, but often times, the meaning will be contained inside a paragraph level (multisentence lever).

3) `Using a multiline splitting`: We can create our own rules not not split by `\n` or stopping characters like `.` but, for example, by headers, subheaders or even multiple lines.

In [0]:
text = """Exhibit 10.2

Execution Version

INTELLECTUAL PROPERTY AGREEMENT

This INTELLECTUAL PROPERTY AGREEMENT (this "Agreement"), dated as of December 31, 2018 (the "Effective Date") is entered into by and between Armstrong Flooring, Inc., a Delaware corporation ("Seller") and AFI Licensing LLC, a Delaware limited liability company ("Licensing" and together with Seller, "Arizona") and AHF Holding, Inc. (formerly known as Tarzan HoldCo, Inc.), a Delaware corporation ("Buyer") and Armstrong Hardwood Flooring Company, a Tennessee corporation (the "Company" and together with Buyer the "Buyer Entities") (each of Arizona on the one hand and the Buyer Entities on the other hand, a "Party" and collectively, the "Parties").

WHEREAS, Seller and Buyer have entered into that certain Stock Purchase Agreement, dated November 14, 2018 (the "Stock Purchase Agreement"); WHEREAS, pursuant to the Stock Purchase Agreement, Seller has agreed to sell and transfer, and Buyer has agreed to purchase and acquire, all of Seller's right, title and interest in and to Armstrong Wood Products, Inc., a Delaware corporation ("AWP") and its Subsidiaries, the Company and HomerWood Hardwood Flooring Company, a Delaware corporation ("HHFC," and together with the Company, the "Company Subsidiaries" and together with AWP, the "Company Entities" and each a "Company Entity") by way of a purchase by Buyer and sale by Seller of the Shares, all upon the terms and condition set forth therein;"""

We will set custom Bounds to False to force ignoring `.` and `\n`, but adding `\n\n` as a paragraph separator

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
paragraphDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("paragraph")\
    .setUseCustomBoundsOnly(False)\
    .setCustomBounds(["\n\n"])\
    .setUseCustomBoundsOnly(True)

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        paragraphDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


In [0]:
lp = LightPipeline(model)

In [0]:
lp.annotate(text)['paragraph']

Out[25]: ['Exhibit 10.2',
 'Execution Version',
 'INTELLECTUAL PROPERTY AGREEMENT',
 'This INTELLECTUAL PROPERTY AGREEMENT (this "Agreement"), dated as of December 31, 2018 (the "Effective Date") is entered into by and between Armstrong Flooring, Inc., a Delaware corporation ("Seller") and AFI Licensing LLC, a Delaware limited liability company ("Licensing" and together with Seller, "Arizona") and AHF Holding, Inc. (formerly known as Tarzan HoldCo, Inc.), a Delaware corporation ("Buyer") and Armstrong Hardwood Flooring Company, a Tennessee corporation (the "Company" and together with Buyer the "Buyer Entities") (each of Arizona on the one hand and the Buyer Entities on the other hand, a "Party" and collectively, the "Parties").',
 'WHEREAS, Seller and Buyer have entered into that certain Stock Purchase Agreement, dated November 14, 2018 (the "Stock Purchase Agreement"); WHEREAS, pursuant to the Stock Purchase Agreement, Seller has agreed to sell and transfer, and Buyer has agreed to purchase and acquire, all of Seller\'s right, title and interest in and to Armstrong Wood Products, Inc., a Delaware corporation ("AWP") and its Subsidiaries, the Company and HomerWood Hardwood Flooring Company, a Delaware corporation ("HHFC," and together with the Company, the "Company Subsidiaries" and together with AWP, the "Company Entities" and each a "Company Entity") by way of a purchase by Buyer and sale by Seller of the Shares, all upon the terms and condition set forth therein;']

In [0]:
lp.annotate(text)['paragraph'][4]

Out[26]: 'WHEREAS, Seller and Buyer have entered into that certain Stock Purchase Agreement, dated November 14, 2018 (the "Stock Purchase Agreement"); WHEREAS, pursuant to the Stock Purchase Agreement, Seller has agreed to sell and transfer, and Buyer has agreed to purchase and acquire, all of Seller\'s right, title and interest in and to Armstrong Wood Products, Inc., a Delaware corporation ("AWP") and its Subsidiaries, the Company and HomerWood Hardwood Flooring Company, a Delaware corporation ("HHFC," and together with the Company, the "Company Subsidiaries" and together with AWP, the "Company Entities" and each a "Company Entity") by way of a purchase by Buyer and sale by Seller of the Shares, all upon the terms and condition set forth therein;'

By headers

In [0]:
text = """
4. GRANT OF KNOW-HOW LICENSE 
4.1 Arizona Know-How Grant. Subject to the terms and conditions of this Agreement, Arizona hereby grants to the Company a perpetual, non- exclusive, royalty-free license in, to and under the Arizona Licensed Know-How for use in the Company Field throughout the world. 
4.2 Company Know-How Grant. Subject to the terms and conditions of this Agreement, the Company hereby grants to Seller a perpetual, non- exclusive, royalty-free license in, to and under the Company Licensed Know-How for use in the Arizona Field throughout the world. 
5. GRANT OF PATENT LICENSE 
5.1 Arizona Patent Grant. Subject to the terms and conditions of this Agreement, Arizona hereby grants to the Company a perpetual, non-exclusive, royalty-free license in, to and under the Arizona Licensed Patents for use in the Company Field throughout the world.
"""

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
paragraphDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("paragraph")\
    .setCustomBounds(["\n\d\. "])\
    .setCustomBoundsStrategy('prepend')\
    .setUseCustomBoundsOnly(True)

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        paragraphDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

lp = LightPipeline(model)
lp.annotate(text)['paragraph']

Out[28]: ['4. GRANT OF KNOW-HOW LICENSE \n4.1 Arizona Know-How Grant. Subject to the terms and conditions of this Agreement, Arizona hereby grants to the Company a perpetual, non- exclusive, royalty-free license in, to and under the Arizona Licensed Know-How for use in the Company Field throughout the world. \n4.2 Company Know-How Grant. Subject to the terms and conditions of this Agreement, the Company hereby grants to Seller a perpetual, non- exclusive, royalty-free license in, to and under the Company Licensed Know-How for use in the Arizona Field throughout the world. \n5. GRANT OF PATENT LICENSE \n5.1 Arizona Patent Grant. Subject to the terms and conditions of this Agreement, Arizona hereby grants to the Company a perpetual, non-exclusive, royalty-free license in, to and under the Arizona Licensed Patents for use in the Company Field throughout the world.']

By subheader

In [0]:
text = """
4. GRANT OF KNOW-HOW LICENSE 
4.1 Arizona Know-How Grant. Subject to the terms and conditions of this Agreement, Arizona hereby grants to the Company a perpetual, non- exclusive, royalty-free license in, to and under the Arizona Licensed Know-How for use in the Company Field throughout the world. 
4.2 Company Know-How Grant. Subject to the terms and conditions of this Agreement, the Company hereby grants to Seller a perpetual, non- exclusive, royalty-free license in, to and under the Company Licensed Know-How for use in the Arizona Field throughout the world. 
5. GRANT OF PATENT LICENSE 
5.1 Arizona Patent Grant. Subject to the terms and conditions of this Agreement, Arizona hereby grants to the Company a perpetual, non-exclusive, royalty-free license in, to and under the Arizona Licensed Patents for use in the Company Field throughout the world.
"""

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
paragraphDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("paragraph")\
    .setCustomBounds(["\n[\d\.]+"])\
    .setCustomBoundsStrategy('prepend')\
    .setUseCustomBoundsOnly(True)

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        paragraphDetector])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

lp = LightPipeline(model)
lp.annotate(text)['paragraph']

Out[30]: ['4. GRANT OF KNOW-HOW LICENSE',
 '4.1 Arizona Know-How Grant. Subject to the terms and conditions of this Agreement, Arizona hereby grants to the Company a perpetual, non- exclusive, royalty-free license in, to and under the Arizona Licensed Know-How for use in the Company Field throughout the world.',
 '4.2 Company Know-How Grant. Subject to the terms and conditions of this Agreement, the Company hereby grants to Seller a perpetual, non- exclusive, royalty-free license in, to and under the Company Licensed Know-How for use in the Arizona Field throughout the world.',
 '5. GRANT OF PATENT LICENSE',
 '5.1 Arizona Patent Grant. Subject to the terms and conditions of this Agreement, Arizona hereby grants to the Company a perpetual, non-exclusive, royalty-free license in, to and under the Arizona Licensed Patents for use in the Company Field throughout the world.']

# 3. Headers and Subheaders
We have a model which is able to understand headers and subheaders in legal documents.

It's an NER model we can use at sentence level, to detect if there is a header (X. HEADER) or a subheader (X.X. Subheader) in a text.

We can use then the headers and subheaders to infer the sections of the document, as shown in the code below.

In [0]:
documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")
        
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

tokenizer = Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

embeddings = BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner_model = finance.NerModel.pretrained("finner_headers", "en", "finance/models")\
        .setInputCols(["sentence", "token", "embeddings"])\
        .setOutputCol("ner")

ner_converter = NerConverter()\
        .setInputCols(["sentence","token","ner"])\
        .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
finner_headers download started this may take some time.
[ | ][ / ][OK!]

In [0]:
text = """PART I
ITEM 1. BUSINESS.
Company Overview
Inuvo is a technology company that develops and sells information technology solutions for marketing.
ITEM 1A. RISK FACTORS.
An investment in our common stock involves a significant degree of risk. 
ITEM 1B. UNRESOLVED STAFF COMMENTS.
Not applicable to a smaller reporting company.
ITEM 2. PROPERTIES.
Our corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021
Angeles, CA, San Jose, CA and Secaucus, NJ.
ITEM 3. LEGAL PROCEEDINGS.
We are not party to any pending legal proceedings.
ITEM 4. MINE SAFETY DISCLOSURES.
Not applicable.
PART II
ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES.
Market Information"""

In [0]:
empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)
lp = LightPipeline(model)

In [0]:
res = lp.fullAnnotate(text)[0]['ner_chunk'] # Document 0
res

Out[34]: [Annotation(chunk, 0, 5, PART I, {'entity': 'HEADER', 'sentence': '0', 'chunk': '0', 'confidence': '0.98364997'}),
 Annotation(chunk, 7, 22, ITEM 1. BUSINESS, {'entity': 'HEADER', 'sentence': '0', 'chunk': '1', 'confidence': '0.99775004'}),
 Annotation(chunk, 25, 40, Company Overview, {'entity': 'SUBHEADER', 'sentence': '1', 'chunk': '2', 'confidence': '0.77835'}),
 Annotation(chunk, 144, 150, ITEM 1A, {'entity': 'SUBHEADER', 'sentence': '3', 'chunk': '3', 'confidence': '0.97815'}),
 Annotation(chunk, 153, 164, RISK FACTORS, {'entity': 'SUBHEADER', 'sentence': '4', 'chunk': '4', 'confidence': '0.97275'}),
 Annotation(chunk, 241, 274, ITEM 1B. UNRESOLVED STAFF COMMENTS, {'entity': 'SUBHEADER', 'sentence': '6', 'chunk': '5', 'confidence': '0.9806166'}),
 Annotation(chunk, 324, 329, ITEM 2, {'entity': 'HEADER', 'sentence': '8', 'chunk': '6', 'confidence': '0.99310005'}),
 Annotation(chunk, 332, 341, PROPERTIES, {'entity': 'SUBHEADER', 'sentence': '9', 'chunk': '7', 'confidence': '0.825'}),
 Annotation(chunk, 582, 588, ITEM 3., {'entity': 'HEADER', 'sentence': '11', 'chunk': '8', 'confidence': '0.7949667'}),
 Annotation(chunk, 590, 606, LEGAL PROCEEDINGS, {'entity': 'SUBHEADER', 'sentence': '12', 'chunk': '9', 'confidence': '0.94984996'}),
 Annotation(chunk, 660, 690, ITEM 4. MINE SAFETY DISCLOSURES, {'entity': 'HEADER', 'sentence': '14', 'chunk': '10', 'confidence': '0.9999833'}),
 Annotation(chunk, 693, 706, Not applicable, {'entity': 'SUBHEADER', 'sentence': '15', 'chunk': '11', 'confidence': '0.53925'}),
 Annotation(chunk, 709, 715, PART II, {'entity': 'HEADER', 'sentence': '16', 'chunk': '12', 'confidence': '0.9721'}),
 Annotation(chunk, 717, 832, ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES, {'entity': 'HEADER', 'sentence': '16', 'chunk': '13', 'confidence': '0.9997334'}),
 Annotation(chunk, 835, 852, Market Information, {'entity': 'SUBHEADER', 'sentence': '17', 'chunk': '14', 'confidence': '0.96634996'})]

In [0]:
sections = []
for ann in res:
  sections.append( (ann.begin, ann.end) )
sections

Out[35]: [(0, 5),
 (7, 22),
 (25, 40),
 (144, 150),
 (153, 164),
 (241, 274),
 (324, 329),
 (332, 341),
 (582, 588),
 (590, 606),
 (660, 690),
 (693, 706),
 (709, 715),
 (717, 832),
 (835, 852)]

In [0]:
section_texts = []
last_section = 0
last_section_name = ""
for s in sections:
  t = last_section_name + text[last_section:s[0]]
  if t != '':
    section_texts.append(t)  
  last_section = s[1]+1
  last_section_name = text[s[0]:s[1]+1]
section_texts.append(last_section_name + text[last_section:s[0]])  


In [0]:
OKGREEN = '\033[92m'
ENDC = '\033[0m'

In [0]:
for t in section_texts:
  print(f"{OKGREEN}SECTION:{ENDC}\n{t}")

SECTION: 
PART I

 SECTION: 
ITEM 1. BUSINESS.

 SECTION: 
Company Overview
Inuvo is a technology company that develops and sells information technology solutions for marketing.

 SECTION: 
ITEM 1A. 
 SECTION: 
RISK FACTORS.
An investment in our common stock involves a significant degree of risk. 

 SECTION: 
ITEM 1B. UNRESOLVED STAFF COMMENTS.
Not applicable to a smaller reporting company.

 SECTION: 
ITEM 2. 
 SECTION: 
PROPERTIES.
Our corporate headquarters are located in Little Rock, Arkansas where we entered into a five-year agreement to lease office space on October 1, 2015 and amended the lease as of February 1, 2021
Angeles, CA, San Jose, CA and Secaucus, NJ.

 SECTION: 
ITEM 3. 
 SECTION: 
LEGAL PROCEEDINGS.
We are not party to any pending legal proceedings.

 SECTION: 
ITEM 4. MINE SAFETY DISCLOSURES.

 SECTION: 
Not applicable.

 SECTION: 
PART II

 SECTION: 
ITEM 5. MARKET FOR REGISTRANT’S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES.

 SECTION: 
Market Information